In [ ]:
import easyocr
import streamlit as st
import re
import numpy as np
import pandas as pd
from PIL import Image

st.title("BizCardX: Extracting Business Card Data with OCR")
primaryColor = '#FF8C02' # Bright Orange

backgroundColor = '#00325B' # Dark Blue

secondaryBackgroundColor = '#55B2FF' # Lighter Blue
uploaded_files = st.file_uploader("Choose a business card")
if uploaded_files is not None:
                # reading the data
        #if uploaded_files is not None:
        image = Image.open(uploaded_files)
        reader = easyocr.Reader(['en'])
        result = reader.readtext(np.array(image), detail = 0)

        st.image(uploaded_files, caption='Business Card')

        dict = {}  # creating a dictionary to store the extracted data

        # creating empty lists to store the data according to the categories
        mobile_number = []
        website = []
        a = []
        company_name = []

        # identifying name and designation from the extracted data
        dict["Name"] = result[0]
        dict["Designation"] = result[1]
        for i in dict.values():
            result.remove(i)

        # using regular functions to identify the data
        for i in result:
            if re.match("^[+]*[(]{0,1}[0-9]{1,4}[)]{0,1}[-\s\./0-9]*$", i) and len(i) >= 10:
                mobile_number.append(i)
            elif re.match(r"^[a-zA-Z0-9+_.-]+@[a-zA-Z0-9.-]+$", i):
                dict["Email_address"] = i
            elif re.match(r"[a-zA-Z].+[a-zA-Z0-9](?:[\.]+[a-z]{2,3})", i) or i[0:3] == "www" or i[0:3] == "WWW":
                website.append(i)
            elif re.match(r"[0-9]{6}", i):
                dict["Pincode"] = i
            elif re.match(r"[a-zA-Z]+\s[0-9]{6}", i):
                a = i.split(" ")
                dict["pincode"] = a[1]
                address2 = a[0]
                address = address + address2
            elif re.match(r"[0-9]{3}\s[a-zA-Z]{3}\s[a-zA-Z]+[\s]?[\,]?[\,]?[\s]?[a-zA-Z]+[\,]?[\;]?[a-zA-Z]+[\,]?[\;]?", i):
                address = i
            elif re.match(r"[a-zA-Z]+", i):
                company_name.append(i)

        # storing combined data which is extracted into the dictionary
        dict["Mobile_Number"] = ",".join(mobile_number)
        dict["Website_url"] = ".".join(website)
        dict["company_name"] = " ".join(company_name)


        # extracting area, city and state from the string address
        def resulted_address(text):
            text = text.split(",")
            for i in text:
                if i == '':
                    text.remove(i)
            dict["Area"] = text[0]
            text.remove(text[0])
            dict["City and State"] = ",".join(text)


        resulted_address(address)

        # streamlit application
        df = pd.DataFrame(data=dict.items(), columns=["details", "data"])
        st.dataframe(df)

        import mysql.connector

        # establishing the connection
        conn = mysql.connector.connect(
            user='root', password='jala007aA@', host='localhost', database='BusinessCard')

        # Creating a cursor object using the cursor() method
        cursor = conn.cursor()
        cursor = conn.cursor(buffered=True)
        #cursor.execute("create database project")
        #cursor.execute("create table card4 (Name varchar(50), Designation varchar(50),Email_address varchar(50), pincode varchar(50),Mobile_Number varchar(50),Website_url varchar(50), company_name varchar(50), Area varchar(50), City_and_State varchar(50))")
        #cursor.execute("create table pics1( image blob)")


        if st.button('save_data'):  # saving the data into database
            cursor.execute("insert into pics values(load_file(image))")
            list2 = [i for i in dict.values()]
            query = "insert into card4 values (%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            val = (list2[0], list2[1], list2[2], list2[3], list2[4], list2[5], list2[6], list2[7], list2[8])
            cursor.execute(query, val)
            conn.commit()

        if st.button('delete_data'):  # button to delete data from the database
            cursor.execute("truncate card4 ")
            cursor.execute("truncate pics1")
        #if st.button('view database'):
             #st.write(cursor.execute("select * from card4"))
             #st.write(cursor.execute("select * from pics1"))

            # print(rows)
             #rows = cursor.fetchall()

        if st.button('view database'):
            cursor.execute("select * from card4")
            result_card4 = cursor.fetchall()
            for row in result_card4:
                st.write(row)
            cursor.execute("select image from pics1")
            result_pics1 = cursor.fetchall()
            cursor.execute("select image from pics1")
            result_pics1 = cursor.fetchall()
            for row in result_pics1:
                img_data = row[0]
                if img_data is not None:
                    img = Image.open(io.BytesIO(img_data))
                    st.image(img)




